In [ ]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.unit_proccessing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = UnitDataProcessing(config)

In [ ]:
self = features_class

In [ ]:
df_item = features_class.df_item


In [ ]:
feature_name = ['f__gps_latitude', 'f__gps_longitude', 'f__gps_accuracy']
data, index_col = self.get_clean_pivot_table(feature_name, remove_low_freq_col=False)

def replace_with_feature_name(columns, feature_names):
    for i, s in enumerate(columns):
        for sub in feature_names:
            if sub in s:
                columns[i] = sub
                break
    return columns

data.columns = replace_with_feature_name(list(data.columns), feature_name)
data = data.reset_index()

In [ ]:
from pyod.models.cof import COF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from sklearn.cluster import DBSCAN, HDBSCAN


model = COF(n_neighbors=20)
#model = CBLOF(contamination=0.01)
#model = KNN()
#model = HDBSCAN()

columns =  ['f__gps_latitude', 'f__gps_longitude'] #['f__gps_latitude', 'f__gps_longitude', 'f__gps_accuracy']#['f__gps_latitude', 'f__gps_longitude'] #
data = data[columns].drop_duplicates()
model.fit(data)

In [ ]:
data[columns].shape, data[columns].drop_duplicates().shape

In [ ]:
data['anomaly'] = model.predict(data[columns])
#data['anomaly'] = model.labels_
#data['anomaly'] = data['anomaly'].apply(lambda x: 1 if x==-1 else 0)
data['anomaly'].value_counts()#, data['anomaly'].value_counts()/data['anomaly'].count()

In [ ]:
import folium
import matplotlib.colors

def get_color(value, vmin, vmax):
    norm = plt.Normalize(vmin, vmax)
    cmap = plt.get_cmap("viridis_r")  # You can use any other colormap
    rgba = cmap(norm(value))
    return matplotlib.colors.rgb2hex(rgba)

In [ ]:
data['anomaly'].value_counts()

In [ ]:
import folium
import matplotlib.colors

def get_color(value, vmin, vmax):
    norm = plt.Normalize(vmin, vmax)
    cmap = plt.get_cmap("viridis_r")  # You can use any other colormap
    rgba = cmap(norm(value))
    return matplotlib.colors.rgb2hex(rgba)
df = data
m = folium.Map(location=[df['f__gps_latitude'].mean(), df['f__gps_longitude'].mean()], zoom_start=10)

vmin = df['anomaly'].min()
vmax = df['anomaly'].max()
#df['color'] = df['s__proximity_counts'].apply(lambda x: 'green' if x>0 else 'blue')
df['color'] = df['anomaly'].apply(lambda x: 'red' if x>0 else 'blue')

for idx, row in df.iterrows():
    color = get_color(row['anomaly'], vmin, vmax)

    marker = folium.CircleMarker(
        location=(row['f__gps_latitude'], row['f__gps_longitude']),
        radius=5,#+row['s__proximity_counts'],
        color=row['color'],
        fill=True,
        #tooltip = row['s__proximity_counts'],
        fill_color=color
    )

    # Add a popup label to display the value of s__spacial_outlier
    folium.Popup(f"Value: {row['anomaly']}").add_to(marker)

    marker.add_to(m)
    
m.save('map_with_labels.html')